# How does SHAP work?

This notebook explores the approach to feature sampling employed in the popular [`shap` python library](https://github.com/slundberg/shap). While the library [and its accompanying NeurIPS paper](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions) are a focal point of the notebook, any promising / insightful directions in the ML, statistics, or game theory literature will be pursued.

# 0. Setup

Load necessary python libraries

In [1]:
import pathlib
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from itertools import combinations, chain
from scipy import stats
from scipy.special import factorial, binom

# 1. Background

## 1.a. Shapley Values

Shapley values have their origins in game theory (Shapley 1953). The basic idea is:
1. $n$ people can play a game
2. I observe the outcomes of all $2^n$ subsets of those $n$ people playing the game

If all I have is \#2 above, can I deduce the value that a specific person contributes on average?

To make this less theoretical, let's consider a simple example. I have 5 people doing a pushup challenge. Let's say they can each do the following amount of pushups in a minute:
1. 100
2. 60
3. 40
4. 30
5. 10

But all I observe is:
* If 1, 2, 3, 4, and 5 all do the challenge, they do 240 total pushups
* If 1, 2, and 4 do the challenge, they do 190 pushups
* If 1 and 3 do the challenge, they do 140 pushups

... and so on for every possible grouping ...

The Shapley value is a formula for backing out an individual's contribution given the observed data:

$\phi_i(x) = \sum_{S \subseteq N} \gamma_S (\nu(S) - \nu(S-i))$

In words, 
* $i$ is the player whose contribution we're interested in
* $S$ is some subset of players
* $\nu(S)$ is the outcome when this subset plays the game
* $\gamma_S$ is a combinatorial weight applied to each subset ($(s-1)!(n-s)!/n!$)

Now, let's generate the "observed" data from the "true" Shapley values so we can see that the Shapley formula recovers the right values

In [2]:
shap_values = np.array([100, 60, 40, 30, 10])
n = len(shap_values)
shap_indices = np.arange(n)
combination_iterator = list(chain.from_iterable(combinations(shap_indices, i) for i in range(n+1)))
value_combinations = np.array([np.sum(shap_values[np.in1d(shap_indices, np.array(comb))]) for comb in combination_iterator])
boolean_matrix = np.array([np.in1d(shap_indices, np.array(comb)) for comb in combination_iterator])*1
my_list = ['Player {:d}'.format(i) for i in range(n)]; my_list.append("value")
observed_data = pd.DataFrame(np.concatenate([boolean_matrix, value_combinations.reshape(-1, 1)], axis=1), columns=my_list)

In [3]:
observed_data

,Player 0,Player 1,Player 2,Player 3,Player 4,value
0,0,0,0,0,0,0
1,1,0,0,0,0,100
2,0,1,0,0,0,60
3,0,0,1,0,0,40
4,0,0,0,1,0,30
5,0,0,0,0,1,10
6,1,1,0,0,0,160
7,1,0,1,0,0,140
8,1,0,0,1,0,130
9,1,0,0,0,1,110


In [4]:
def set_minus(row, item):
    temp = row.copy(); temp[item] = 0
    return temp

def row_equality(row, search_array):
    return np.array_equal(row, search_array)

def row_diff(matrix_row, df, boolean_matrix, item):
    return (df.loc[np.apply_along_axis(row_equality, 1, boolean_matrix, matrix_row), "value"].values - 
            df.loc[np.apply_along_axis(row_equality, 1, boolean_matrix, set_minus(matrix_row, item)), "value"].values)

def shap_difference(boolean_matrix, observed_data, item):
    return np.apply_along_axis(row_diff, 1, boolean_matrix, df=observed_data, boolean_matrix=boolean_matrix, item=item)

def shap_weight(s, n):
    return (factorial(s-1)*factorial(n-s))/factorial(n)

def compute_shap(player_num, boolean_matrix, observed_data, n):
    return np.sum(shap_difference(boolean_matrix, observed_data, player_num).reshape(-1)*shap_weight(s=np.sum(boolean_matrix, axis=1), n = n))

Now, let's compute Shapley values for each of the players in our game and see if they compare favorably to what we know to be the correct answer

In [5]:
for player in shap_indices:
    shap_val = compute_shap(player, boolean_matrix, observed_data, n)
    print("Player {:d} Shapley value: {:.2f}".format(player+1, shap_val))

Player 1 Shapley value: 100.00
Player 2 Shapley value: 60.00
Player 3 Shapley value: 40.00
Player 4 Shapley value: 30.00
Player 5 Shapley value: 10.00


So it works as advertised (though this was a pretty easy example).

Now, the way Shapley values work as a regression explanation method, it would be intractable to do what we just did above. We can't iterate over $2^n$ combinations of features when the feature set is large. (For 15 features this would be 33k combinations.)

## 1.b. Shap library deep dive

How does the sampling in `shap` work? There are a few approaches, mostly designed to explain individual predictions. 

For the most general version of SHAP (the "Kernel Explainer"), the underlying code is [here](https://github.com/slundberg/shap/blob/master/shap/explainers/kernel.py#L322)\[1\]. 

At a high level, it's doing the following:

1. Take user input on:
    1. A data point to be explained (`explanation_data`)
    2. A dataset of "background" or reference values for comparison (`background_data`)
    3. The number of samples to generate (`n_samples`)
2. Narrow down to the features that vary between the background dataset and the prediction of interest (i.e. if a dataset is full of people for whom `city=SF`, don't try to manipulate the `city` feature and attribute prediction changes to it). Call this value $M$.
3. Letting $x$ be the number of features to modify in given sample, iterate over all $x$ from 1 to $M$:
    1. For each of the $M \choose x$ combinations of $x$ features:
        1. Switch the values of the $x$ selected features in the background dataset to their values in the `explanation_data`
        2. Add a corresponding sample with the opposite set of columns flipped (for example, if step 1 was to flip columns 1, 2, and 4, step 2 would be to flip columns 3 and 5)
        3. Once you can no longer iterate through combinations without hitting the `n_samples` limit, take random samples of features to round out the number of sampled data points to `n_samples` exactly
4. Run weighted least squares regression with kernel weights based on the number of features "flipped" (i.e. if all but one of the features is the same between the background sample and the synthetic sample, the weight is smaller as the synthetic sample is "closer" to the original data point).

# 2. Implementation

In general, the `SHAP` approach will return values that sum to the difference of the average model output for the background dataset and that of the prediction to be explained. 

In order to get `SHAP` to return the same shapley values we computed above, we can just set the "background" dataset to one datapoint of all zeros (\[0, 0, 0, 0, 0\]) and set the explanation point to be all ones \[1, 1, 1, 1, 1\].

To build a deeper understanding, the code below is a modified / simplified version of the [shap KernelExplainer code](https://github.com/slundberg/shap/blob/master/shap/explainers/kernel.py).

We start by defining three things to implement this on our own:
1. The number of samples for the program to take
2. The background dataset to compare to the outcome
3. The data point to be explained

In [6]:
n_samples = 20
explanation_index = 31
explanation_data = np.array(observed_data.loc[explanation_index, ~(observed_data.columns.isin(["value"]))]).reshape(1, -1)
explanation_outcome = np.array(observed_data.loc[explanation_index, "value"]).reshape(1, -1)
background_index = 0
background_data = np.array(observed_data.loc[background_index, ~(observed_data.columns.isin(["value"]))]).reshape(1, -1) if np.size(background_index) == 1 else np.array(observed_data.loc[background_index, ~(observed_data.columns.isin(["value"]))])
background_outcome = np.array(observed_data.loc[background_index, "value"]).reshape(1, -1)

Extract some information from these inputs to make processing easier and construct the "weights" needed for weighted least squares regression (used to fit the Shapley values).

In [7]:
# If someone asks for more samples than either the total number of possible combinations (2^p) or 2^32, 
# just ignore the rest
n = np.shape(background_data)[0]
p = np.shape(background_data)[1]
if n_samples >= 2**32:
    n_samples = 2**32 - 2
elif n_samples > 2**p and p < 32:
    n_samples = 2**p - 2

# Loop through as many combinations of the features as possible
synth_data = np.tile(background_data, (n_samples, 1))
columns_flipped = np.zeros((n_samples, p))
column_nums = np.arange(p)
n_subsets = np.int(np.ceil((p-1)/2.0))
n_paired_subsets = np.int(np.floor((p-1)/2.0))
weight_vector = np.array([(p - 1.0) / (i * (p - i)) for i in range(1, n_subsets + 1)])
weight_vector[:n_paired_subsets] *= 2
weight_vector /= np.sum(weight_vector)
kernelWeights = np.zeros(n_samples)
y = np.zeros((n_samples * n, 1))
ey = np.zeros((n_samples, 1))
data_weights = np.ones(n)
data_weights /= np.sum(data_weights)

Build a synthetic dataset according to the sampling procedure above

In [8]:
i = 0
for subset_size in range(1, (n_subsets+1)):
    # Skip over a set of combinations if there's no room to add them all to the synthetic data
    num_combinations = binom(p, subset_size) if subset_size > n_paired_subsets else binom(p, subset_size)*2
    if num_combinations <= (n_samples - i):

        w = weight_vector[subset_size - 1] / num_combinations
        
        # Use itertools to run through these combinations
        for comb in combinations(column_nums, subset_size):
            flip_indicator = np.isin(column_nums, comb)
            flip_columns = np.flatnonzero(flip_indicator)
            columns_flipped[i, ] = flip_indicator
            synth_data[i*n:(i+1)*n, flip_columns] = explanation_data[0, flip_columns]
            kernelWeights[i] = w
            i += 1
            
            # If there is a paired subset on the other side of the combinatorial tree (i.e. 6 choose 5 
            # instead of 6 choose 1), then sample that as well
            if subset_size <= n_paired_subsets:
                flip_indicator = ~np.isin(column_nums, comb)
                flip_columns = np.flatnonzero(flip_indicator)
                columns_flipped[i, ] = flip_indicator
                synth_data[i*n:(i+1)*n, flip_columns] = explanation_data[0, flip_columns]
                kernelWeights[i] = w
                i += 1
                
    elif i < n_samples:
        # Otherwise just take random samples from those combinations to fill up the synth_data set
        comb_list = np.random.permutation(list(combinations(column_nums, subset_size)))
        remaining_loops = (n_samples - i)//2
        w = np.sum(weight_vector[(subset_size-1):])/(n_samples - i)
        for j in range(0, remaining_loops):
            # Pick a random sample
            column_samples = comb_list[j]
            flip_indicator = np.isin(column_nums, column_samples)
            flip_columns = np.flatnonzero(flip_indicator)
            columns_flipped[i, ] = flip_indicator
            synth_data[i*n:(i+1)*n, flip_columns] = explanation_data[0, flip_columns]
            kernelWeights[i] = w
            i += 1

            # Then add its paired converse to the sample list as well
            flip_indicator = ~np.isin(column_nums, column_samples)
            flip_columns = np.flatnonzero(flip_indicator)
            columns_flipped[i, ] = flip_indicator
            synth_data[i*n:(i+1)*n, flip_columns] = explanation_data[0, flip_columns]
            kernelWeights[i] = w
            i += 1

Compute the expected value of the "model" (in this case, it's just a matrix of completely enumerated values) for each of the synthetic data points

In [9]:
# Run the model on the synthetic data
def extract_model_values(matrix_row, df, boolean_matrix):
    return df.loc[np.apply_along_axis(row_equality, 1, boolean_matrix, matrix_row), "value"].values
modelOut = np.apply_along_axis(extract_model_values, 1, synth_data, df = observed_data, boolean_matrix=boolean_matrix).reshape(-1)
y = np.reshape(modelOut, (n_samples * n, 1))

# Calculate expected value for each synthetic datapoint, making possible use of data weights
for i in range(0, n_samples * n):
    eyVal = np.zeros(1)
    for j in range(0, n):
        eyVal += y[i * n + j, :] * data_weights[j]
    ey[i, :] = eyVal

Estimate Shapley values by a weighted least squares regression

In [10]:
# Compute the average value of the outcome in the background dataset
original_value = np.squeeze(np.sum((background_outcome.T * data_weights).T, 0))
# Compute the value of the outcome for the data point we're explaining
explanation_value = np.squeeze(explanation_outcome)
# Look at the difference between the model estimates for the synthetic dataset and the background average
eyAdj = np.squeeze(ey) - original_value

# # Note: when there are a lot of features, the SHAP library will do some sort of 
# regularization here to narrow to a smaller set of features
# # Skipping that for simplicity

# Only need to estimate values for p-1 columns, since the total sum of Shapley values is 
# already constrained to be explanation_value - original_value
eyAdj2 = eyAdj - columns_flipped[:, column_nums[-1]] * (explanation_value - original_value)
etmp = np.transpose(np.transpose(columns_flipped[:, column_nums[:-1]]) - columns_flipped[:, column_nums[-1]])

# Estimate shapley values using weighted least squares
tmp = np.transpose(np.transpose(etmp) * np.transpose(kernelWeights))
tmp2 = np.linalg.inv(np.dot(np.transpose(tmp), etmp))
w = np.dot(tmp2, np.dot(np.transpose(tmp), eyAdj2))
phi = np.zeros(p)
phi[column_nums[:-1]] = w
phi[column_nums[-1]] = (explanation_value - original_value) - sum(w)

Look at estimated shapley values for that sample

In [11]:
print("Average outcome in background dataset: {:.2f}".format(original_value))
print("Outcome for the explanation data point: {:.2f}".format(explanation_value))
print("Difference in outcomes: {:.2f}".format(explanation_value-original_value))
for player in shap_indices:
    print("X{:d} data value: {:.2f}".format(player+1, np.squeeze(explanation_data)[player]))
    shap_val = phi[player]
    print("X{:d} Shapley value: {:.2f}".format(player+1, shap_val))

Average outcome in background dataset: 0.00
Outcome for the explanation data point: 240.00
Difference in outcomes: 240.00
X1 data value: 1.00
X1 Shapley value: 100.00
X2 data value: 1.00
X2 Shapley value: 60.00
X3 data value: 1.00
X3 Shapley value: 40.00
X4 data value: 1.00
X4 Shapley value: 30.00
X5 data value: 1.00
X5 Shapley value: 10.00


So that worked as well

# Notes

\[1\] There are technically a few different "explainer" implementations contained within shap (one optimized for trees, one for deep learning, etc...), but the code linked above pertains to the most general (model-agnostic) explanation method

# References

Shapley, L. S. "A Value for n-Person Games," Annals of Math Studies, 28(1953),307-317.